In [2]:
import mysql.connector
import pandas as pd


config = {
    'user': 'root',
    'password': '*****',
    'host': 'localhost',
    'database': 'PROJECTS1'
}
connection = mysql.connector.connect(**config)
cursor = connection.cursor()


columnas=(pd.read_csv('C:/Users/rober/Desktop/bbdd/Iowa_Liquor_Sales.csv',dtype="str",low_memory=True,nrows=1)).columns
chunk=10**4

for df1 in pd.read_csv('C:/Users/rober/Desktop/bbdd/Iowa_Liquor_Sales.csv',low_memory=False,chunksize=chunk, dtype=str,usecols=columnas):

    df1['Date'] = pd.to_datetime(df1['Date'])
    df1['Date'] = df1['Date'].dt.strftime('%Y-%m-%d')
    df1['Date'] = df1['Date'].astype(str)

    cols_mod=['State Bottle Cost','State Bottle Retail','Sale (Dollars)']
    for col in cols_mod:
        df1[col] = df1[col].str.replace('$','')

    for i in range(df1.shape[0]):
        query = """
        INSERT INTO LIQUOR_SALES (Invoice,Date,Store_Number,Store_Name,Address,
        City,Zip_Code,Store_Location,County_Number,County,Category,Category_Name,
        Vendor_Number,Vendor_Name,Item_Number,Item_Description,Pack,Bottle_Volume_ml,
        State_Bottle_Cost,State_Bottle_Retail,Bottles_Sold,Sale_Dollars,Volume_Sold_Liters,Volume_Sold_Gallons) 
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """

        list=df1.iloc[i].tolist()
        list[17]=int(list[17])
        list[18]=float(list[18])
        list[19]=float(list[19])
        list[20]=int(list[20])
        list[21]=float(list[21])
        list[22]=float(list[22])
        list[23]=float(list[23])
        values = tuple(list)

        cursor.execute(query,values)
        connection.commit()

cursor.close()
connection.close()

#130min
